# Data Cleaning and Processing

In [1]:
import pandas as pd
import numpy as np
import os
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import MinMaxScaler
import gender_guesser.detector as gender

## Load datasets
Load all 6 datasets (true, mostly-true, half-true, barely-true, false, pants-fire) and combine them into a single one.

In [2]:
# Read in and append dataframes
data_path = 'data/'
df = pd.DataFrame()
for filename in os.listdir(data_path):
     df = df.append(pd.read_csv(data_path + filename, dtype={'label':str}, na_values='Unspecified'), ignore_index=True)

df.head()

,label,quote,context,author,date,categories,staff
0,barely-true,“Pennsylvania just banned alcohol sales.”,a Facebook post,Viral image,"November 24, 2020","Facebook Fact-checks, Coronavirus",Ciara O'Rourke
1,barely-true,"“666,000 teachers have been laid off already s...",a virtual roundtable,Joe Biden,"November 18, 2020","Education, Coronavirus",Bill McCarthy
2,barely-true,“David Perdue says he'll do everything in his ...,an ad,Jon Ossoff,"November 17, 2020","Georgia, Negative Campaigning",Tom Kertscher
3,barely-true,Says “47 additional counties used the same sof...,a Facebook post,Ted Nugent,"November 17, 2020","Elections, Facebook Fact-checks",Samantha Putterman
4,barely-true,"""Voter FRAUD exposed in Georgia. Over 2600 vot...",in a Live video,Facebook posts,"November 16, 2020","Georgia, Elections, Facebook Fact-checks",Daniel Funke


## Load metadata
Load additional information on authors and merge to main dataset.

In [3]:
# Read in meta data
metadata_path = 'metadata/'
df_personalities = pd.read_csv(metadata_path + 'personalities.csv')
df_personalities.head()

,personality,affiliation,description,link
0,13th District GOP slate,Republican,The 13th District GOP slate includes state Sen...,NaN
1,18% of the American public,None,NaN,NaN
2,60 Plus Association,None,The 60 Plus Association is a conservative advo...,http://www.60plus.org/
3,AARP,None,"AARP is a nonprofit, nonpartisan organization ...",http://www.aarp.org/
4,Greg Abbott,Republican,Greg Abbott won election as governor of Texas ...,http://gregabbott.com/


### Deduplicate Entries
Ensure that there are no duplicated entries.

In [4]:
dup_personalities = df_personalities['personality'][df_personalities['personality'].duplicated()].unique()
df_personalities.loc[df_personalities['personality'].isin(dup_personalities)].drop_duplicates()

,personality,affiliation,description,link
210,Don Bacon,Republican,Don Bacon is the Republican U.S. Rep. for Nebr...,NaN
236,Lou Barletta,Republican,Lou Barletta is a Republican U.S. Congressman ...,https://barletta.house.gov/
708,Gil Cisneros,Democrat,Democratic candidate Gil Cisneros is running i...,NaN
719,Citizens to Protect Transportation Funding,Organization,Citizens to Protect Transportation Funding is ...,NaN
741,CleanTV,None,A politically-charged site that claims to offe...,NaN
775,Mike Coffman,Republican,Mike Coffman is a member of the U.S. House of ...,http://coffman.house.gov/
805,Colorado Citizens for Accountable Government,None,Colorado Citizens for Accountable Government i...,NaN
1359,Florida Democrats,Organization,Florida Democratic Party,NaN
1525,E. Gordon Gee,None,Has been president of Ohio State University si...,http://president.osu.edu/
1757,Mark Harris,Democrat,"Mark Harris, a Wisconsin Democrat, served as m...",http://www.co.winnebago.wi.us/county-executive


### Derive Author's Gender
Using `gender-guesser`, we attempt to derive additional information of the author's gender based on their **first** names. Each author is classied into either `male`, `female`, `mostly male` and `mostly female`, whereas entries that are not names (ie. organizations, entities, etc.) are assigned `unknown`.

In [5]:
# Derive gender from first name whenever possible
gd = gender.Detector()
unk_word_list = ['The', 'Young', 'My', 'In', 'Ban', 'Free'] # List of first words known to be misclassified
unk_list = ['Al Jazeera America', 'Austin Board of Realtors PAC', 'Austin Fund for Quality Healthcare', 'Austin Independent School District', 'Austin Water Utility', 'Austin for a Better Future', 'Brady Campaign to Prevent Gun Violence', 'Christian Broadcasting Network', 'Clayton County Government', 'Clayton County Schools', 'Dane County Republican Party', 'Dustin Inman Society', 'Elle', 'Fair Districts Florida', 'Forbes blog', 'Georgia Association of Homes and Services for Children', 'Georgia Association of Latino Elected Officials', 'Georgia Craft Brewers Guild', 'Georgia Democrats', 'Georgia Department of Economic Development', 'Georgia Department of Public Health', 'Georgia Department of Transportation', 'Georgia Family Council', 'Georgia Farm Bureau', 'Georgia Green Party', 'Georgia Gun Owners', 'Georgia House Democratic Caucus on behalf of Elena Parent', 'Georgia Lottery', 'Georgia Restaurant Association', 'Georgia State Road and Tollway Authority', 'Georgia Voice', 'Georgia politicians', 'Georgia state senators', 'Gun Free UT Gun Free UT', 'Gun Owners of America', 'Hal Turner Radio Show', 'Save America\'s Postal Service', 'Save Flexible Spending Plans', 'Save My Care', 'Save Our City, Milwaukeeans Can\'t Wait', 'Save Our Springs Alliance', 'Sierra Club', 'Travis County Republican Party', 'Urban Intellectuals', 'Virginia Center for Public Safety', 'Virginia Education Association', 'Virginia First Foundation', 'Virginia House Democratic Caucus' ,'Virginia Interfaith Center for Public Policy', 'Virginia Lottery', 'Virginia Senate Democratic Caucus', 'Virginia Senate Democrats', 'Virginia Society for Human Life', 'Virginia Tea Party Patriots']
df_personalities['gender'] = df_personalities['personality'].apply(lambda x: gd.get_gender(x.split()[0]))
df_personalities['gender'] = np.where(df_personalities['personality'].str.split().str[0].isin(unk_word_list), 'unknown', df_personalities['gender'])
df_personalities['gender'] = np.where(df_personalities['personality'].isin(unk_list), 'unknown', df_personalities['gender'])
df_personalities['gender'].value_counts()

male             2311
unknown          1320
female            707
mostly_male       176
mostly_female      96
andy               29
Name: gender, dtype: int64

### Correct Misclassified Genders
We leverage the description of each author to correct some of the misclassified genders by checking to see if there are contradictions between the assigned gender and the description. For instance, if an author is assigned the gender `male`, but the description only contains `she` and `her`, there are high chances that it is a misclassification. 

In [6]:
# List of known males and females (hardcoded due to exceptions in following rules)
male_list = ['Mike Martinez', 'Matt Tighe']
female_list = ['Marco Rubio\'s heckler', 'Sharron Angle', 'Ann Marie Buerkle', 'Kaya Jones', 'Jane O’Meara Sanders', 'Tiffany Trump', 'Lauren Kane', 'María Teresa Kumar', 'Rick Scott\'s Starbucks heckler']

tmp = df_personalities.copy()

# Boolean variables to indicate presence of he/his or she/her in description
tmp['male_check'] = tmp['description'].apply(lambda x: np.nan if pd.isnull(x) else any(word in x.lower().split() for word in ['he', 'his']))
tmp['female_check'] = tmp['description'].apply(lambda x: np.nan if pd.isnull(x) else any(word in x.lower().split() for word in ['she', 'her']))

# Correct mis-classified records by generating lists of authors who should actually be male or female
# Logic: ie. if classified as male, but description only contains she/her, high probability is female
m2f = [x for x in tmp.loc[(tmp['gender'] == 'male') & (tmp['male_check'] == False) & (tmp['female_check'] == True), 'personality'] if x not in male_list]
f2m = [x for x in tmp.loc[(tmp['gender'] == 'female') & (tmp['male_check'] == True) & (tmp['female_check'] == False), 'personality'] if x not in female_list]
mm2f = [x for x in tmp.loc[(tmp['gender'] == 'mostly_male') & (tmp['male_check'] == False) & (tmp['female_check'] == True), 'personality'] if x not in male_list]
mm2m = [x for x in tmp.loc[(tmp['gender'] == 'mostly_male') & (tmp['male_check'] == True) & (tmp['female_check'] == False), 'personality'] if x not in female_list]
mf2f = [x for x in tmp.loc[(tmp['gender'] == 'mostly_female') & (tmp['male_check'] == False) & (tmp['female_check'] == True), 'personality'] if x not in male_list]
mf2m = [x for x in tmp.loc[(tmp['gender'] == 'mostly_female') & (tmp['male_check'] == True) & (tmp['female_check'] == False), 'personality'] if x not in female_list]
a2m = [x for x in tmp.loc[(tmp['gender'] == 'andy') & (tmp['male_check'] == True) & (tmp['female_check'] == False), 'personality'] if x not in female_list]
a2f = [x for x in tmp.loc[(tmp['gender'] == 'andy') & (tmp['male_check'] == False) & (tmp['female_check'] == True), 'personality'] if x not in male_list]

# Correct mis-classified genders 
df_personalities['gender'] = np.where(df_personalities['personality'].isin(male_list + f2m + mm2m + mf2m + a2m), 'male', 
                                      np.where(df_personalities['personality'].isin(female_list + m2f + mm2f + mf2f + a2f), 'female', 
                                          df_personalities['gender']))

df_personalities['gender'].value_counts()

male             2405
unknown          1320
female            751
mostly_male        94
mostly_female      52
andy               17
Name: gender, dtype: int64

### Merge metadata

In [7]:
# Merge meta data
df = df.merge(df_personalities, left_on='author', right_on='personality', how='left')
df.drop('personality', axis=1, inplace=True)

In [8]:
df.loc[df['author'] == 'Mark Harris']

,label,quote,context,author,date,categories,staff,affiliation,description,link,gender
527,barely-true,"Liberal Democrats"" who back Dan McCready ""cut ...",NaN,Mark Harris,NaN,"Health Care, North Carolina",Rashaan Ayesh,Democrat,"Mark Harris, a Wisconsin Democrat, served as m...",http://www.co.winnebago.wi.us/county-executive,male
528,barely-true,"Liberal Democrats"" who back Dan McCready ""cut ...",NaN,Mark Harris,NaN,"Health Care, North Carolina",Rashaan Ayesh,Republican,Rev. Mark Harris is the former pastor of First...,https://www.markharrisforcongress.com/,male
4465,false,"In our debate, Dan McCready doubled down on hi...",a tweet,Mark Harris,"October 10, 2018","Impeachment, North Carolina",Paul Specht,Democrat,"Mark Harris, a Wisconsin Democrat, served as m...",http://www.co.winnebago.wi.us/county-executive,male
4466,false,"In our debate, Dan McCready doubled down on hi...",a tweet,Mark Harris,"October 10, 2018","Impeachment, North Carolina",Paul Specht,Republican,Rev. Mark Harris is the former pastor of First...,https://www.markharrisforcongress.com/,male
4662,false,"Rep. Robert Pittenger ""stood against funding f...",a Facebook post,Mark Harris,"March 29, 2018","Immigration, North Carolina",Paul Specht,Democrat,"Mark Harris, a Wisconsin Democrat, served as m...",http://www.co.winnebago.wi.us/county-executive,male
4663,false,"Rep. Robert Pittenger ""stood against funding f...",a Facebook post,Mark Harris,"March 29, 2018","Immigration, North Carolina",Paul Specht,Republican,Rev. Mark Harris is the former pastor of First...,https://www.markharrisforcongress.com/,male
4685,false,"Rep. Pittenger is ""constantly rated as one of ...",a newsletter,Mark Harris,"February 9, 2018","Congress, North Carolina",Paul Specht,Democrat,"Mark Harris, a Wisconsin Democrat, served as m...",http://www.co.winnebago.wi.us/county-executive,male
4686,false,"Rep. Pittenger is ""constantly rated as one of ...",a newsletter,Mark Harris,"February 9, 2018","Congress, North Carolina",Paul Specht,Republican,Rev. Mark Harris is the former pastor of First...,https://www.markharrisforcongress.com/,male
7633,half-true,Says Democratic North Carolina House candidate...,A campaign ad,Mark Harris,"September 20, 2018","Bipartisanship, North Carolina",Stephen Koff,Democrat,"Mark Harris, a Wisconsin Democrat, served as m...",http://www.co.winnebago.wi.us/county-executive,male
7634,half-true,Says Democratic North Carolina House candidate...,A campaign ad,Mark Harris,"September 20, 2018","Bipartisanship, North Carolina",Stephen Koff,Republican,Rev. Mark Harris is the former pastor of First...,https://www.markharrisforcongress.com/,male


In [9]:
df_personalities.loc[df_personalities['gender'] == 'female', 'personality'].sort_values()

3906          Abigail Spanberger
1163            Ainsley Earhardt
974              Alberta Darling
3836                   Alex Sink
2989    Alexandria Ocasio-Cortez
                  ...           
984                  Wendy Davis
2426                  Wendy Long
30                Yolanda Adrean
2592          Yvette McGee Brown
2418                 Zoe Lofgren
Name: personality, Length: 751, dtype: object

## Derive Additional Features

In [10]:
# Date related features
df['date_formatted'] = pd.to_datetime(df['date'], format='%B %d, %Y')
df['year'] = df['date_formatted'].dt.year.astype('Int64')
df['month'] = df['date_formatted'].dt.month.astype('Int64')
df['day'] = df['date_formatted'].dt.day.astype('Int64')

In [11]:
# Quote related features
df['num_words'] = df['quote'].str.split().str.len()
df['num_chars'] = df['quote'].str.len()
df['avg_word_len'] = df['quote'].apply(lambda x: round((sum(len(word) for word in x.split()) / len(x.split())), 1))
df['num_stopwords'] = df['quote'].apply(lambda x: len([w for w in x.split() if w.lower() in stopwords.words('english')]))

In [12]:
# Normalize numerical variables
scaler = MinMaxScaler()
features = ['num_words', 'num_chars', 'avg_word_len', 'num_stopwords']
for feat in features:
    df[[feat]] = scaler.fit_transform(df[[feat]])

In [13]:
df.head()

,label,quote,context,author,date,categories,staff,affiliation,description,link,gender,date_formatted,year,month,day,num_words,num_chars,avg_word_len,num_stopwords
0,barely-true,“Pennsylvania just banned alcohol sales.”,a Facebook post,Viral image,"November 24, 2020","Facebook Fact-checks, Coronavirus",Ciara O'Rourke,None,"Graphics, pictures and charts shared on social...",NaN,unknown,2020-11-24,2020,11,24,0.038961,0.060606,0.657143,0.022727
1,barely-true,"“666,000 teachers have been laid off already s...",a virtual roundtable,Joe Biden,"November 18, 2020","Education, Coronavirus",Bill McCarthy,Democrat,Joe Biden is President-elect of the United Sta...,https://www.joebiden.com/,male,2020-11-18,2020,11,18,0.090909,0.100233,0.400000,0.068182
2,barely-true,“David Perdue says he'll do everything in his ...,an ad,Jon Ossoff,"November 17, 2020","Georgia, Negative Campaigning",Tom Kertscher,Democrat,Jon Ossoff is a Democrat running to succeed fo...,https://electjon.com/,male,2020-11-17,2020,11,17,0.168831,0.156177,0.242857,0.090909
3,barely-true,Says “47 additional counties used the same sof...,a Facebook post,Ted Nugent,"November 17, 2020","Elections, Facebook Fact-checks",Samantha Putterman,Republican,"Ted Nugent, who lives near Waco, performed aft...",http://www.tednugent.com/,male,2020-11-17,2020,11,17,0.415584,0.449883,0.328571,0.295455
4,barely-true,"""Voter FRAUD exposed in Georgia. Over 2600 vot...",in a Live video,Facebook posts,"November 16, 2020","Georgia, Elections, Facebook Fact-checks",Daniel Funke,None,Posters on Facebook and other social media net...,https://www.facebook.com/,unknown,2020-11-16,2020,11,16,0.090909,0.100233,0.400000,0.045455
